In [12]:
#basic library 
import pandas as pd                                                       
import numpy as np                                                                                                
import re                                                                   
import joblib           
import string                                                                
#NLTK Corner 
import nltk                                                                
from nltk.corpus import stopwords                                                                           
from nltk.stem import PorterStemmer                                          
from nltk.tokenize import sent_tokenize, word_tokenize                                                                                          
#Tensorflow Corner 
import tensorflow as tf
from sklearn import model_selection                                                       
from tensorflow.keras.preprocessing.text import Tokenizer                     
from tensorflow.keras.preprocessing.sequence import pad_sequences             
#Sklearn gang                                                                
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer                   
from sklearn.model_selection import train_test_split                          
from sklearn.feature_extraction.text import TfidfVectorizer                   
#Keras club 
from keras.models import Sequential                                           
from keras import layers                                                     
from tensorflow import keras                                                  
from keras.preprocessing import sequence                                      
from keras.layers import Dense,Flatten, Dropout, Embedding, LSTM, SpatialDropout1D, Conv2D,TimeDistributed, MaxPool1D, Conv1D,GlobalMaxPool1D
import tensorflow as tf
import tensorflow_addons as tfa

import gensim
from gensim.models import Word2Vec

In [13]:
X = pd.read_csv('X.csv')
Y = pd.read_csv('Y.csv')
Y = Y.drop("Unnamed: 0", axis='columns')


In [14]:
MAX_NB_WORDS = 256
MAX_SEQUENCE_LENGTH = 150 
EMBEDDING_DIM = 64

In [15]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index
X = tokenizer.texts_to_sequences(X)

# print('Found %s unique tokens.' % len(word_index))

In [16]:
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
# print('Shape of data tensor:', X.shape)

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.1)

# Modeling

In [18]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=512, kernel_size=5, input_shape=(X_train.shape[1],1), activation='relu'))
model.add(MaxPool1D(pool_size=5))
# model.add(Dropout(0.7))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(16, activation='softmax'))
model.compile(loss=tf.keras.losses.BinaryFocalCrossentropy(), optimizer= tf.keras.optimizers.Adam(learning_rate=0.001),metrics=tf.keras.metrics.top_k_categorical_accuracy)

In [19]:
acc = []; loss= []
for item in range(3) :
    epochs = 10
    model.fit(
        X_train, Y_train,
        batch_size=32,
        epochs=epochs,
        verbose=1,
        validation_data = (X_test, Y_test))
    item = item + 1
    y_pred = model.predict(X_test)
    score = (model.evaluate(X_test, Y_test, batch_size=32, verbose=1))
    acc.append(score[1])
    metric = tfa.metrics.HammingLoss(mode='multilabel', threshold=0.1)
    metric.update_state(Y_test, y_pred)
    loss.append(metric.result().numpy())

    
print(acc)
print(loss)

Epoch 1/10
176/176 [==============================] - 22s 104ms/step - loss: 0.0222 - top_k_categorical_accuracy: 0.6990 - val_loss: 0.0143 - val_top_k_categorical_accuracy: 0.8958
Epoch 2/10
176/176 [==============================] - 18s 100ms/step - loss: 0.0159 - top_k_categorical_accuracy: 0.7749 - val_loss: 0.0138 - val_top_k_categorical_accuracy: 0.8942
Epoch 3/10
176/176 [==============================] - 18s 100ms/step - loss: 0.0148 - top_k_categorical_accuracy: 0.8243 - val_loss: 0.0134 - val_top_k_categorical_accuracy: 0.8974
Epoch 4/10
176/176 [==============================] - 18s 101ms/step - loss: 0.0141 - top_k_categorical_accuracy: 0.8409 - val_loss: 0.0133 - val_top_k_categorical_accuracy: 0.9054
Epoch 5/10
176/176 [==============================] - 18s 102ms/step - loss: 0.0137 - top_k_categorical_accuracy: 0.8526 - val_loss: 0.0131 - val_top_k_categorical_accuracy: 0.9071
Epoch 6/10
176/176 [==============================] - 18s 104ms/step - loss: 0.0133 - top_k_cat

In [9]:
# model.save('clstm_0.098_89.7.h5')

In [20]:
sum(loss)/3

0.10122863203287125